In [ ]:
!pip uninstall tensorflow -y
!pip  install transformers==4.22.1 -q
!pip install evaluate

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
import load_data
from load_data import Data

In [ ]:
import transformers
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, \
     DataCollatorWithPadding, pipeline
from transformers import DataCollatorWithPadding
import evaluate

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from datasets.dataset_dict import DatasetDict

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
def load_dataset(path):
    #Load a datafraom from the Data class from load_data
    data = Data(path)
    data.handle_file()
    data.convert_json_to_dataframe()
    data.get_next_value()
    data.compare_values()
    data.label_sentences()
    data.initial_df()
    return data

In [ ]:
def label_tokens(token_lists, seq):
    #Labeling as 1 only for the token, which is the last token of the setence labeled as 1
    t_l = []
    for i, t in enumerate(range(len(token_lists))):
        if seq !=0 and i == len(token_lists)-1:
            t_l.append(test_label[seq])
        else :
            t_l.append(0)
    return t_l

In [ ]:
def flatten_list(nested_list):
    #Changing the nested list to a one-dimensional list express
    flat_list = []
    for sublist in nested_list:
        flat_list.extend(sublist)
    return flat_list

In [ ]:
def split_token_sentences(list_sentence):
    #Splitting sentence to tokens
    #Labeling as 1 only for the token, which is the last token of the setence labeled as 1

    s = []
    t_s = []
    t_l = []
    i=0
    for seq, text in enumerate(list_sentence):
        for match in re.finditer(r"\,|\.|\;|\?|\!", text):
            a = text[i:match.start()+1]
            a.strip()
            b = a.split()
            s.append(a)
            t_s.append(b)
            if match.start() != len(text) - 1:
                i = match.start()+1
                t_l.append(label_tokens(b, 0))
            else:
                i=0
                t_l.append(label_tokens(b, seq))
    return s, t_s, t_l

In [ ]:
def generate_test_dataset(tokenized_text, labels):
    #Generating sequences to generate different sequences of speaking by combining tokens
    #Labeling sequences to identify the moments when each speaker takes their turn

    test_seq_tokens= []
    test_seq_labels= []
    t=''
    s=0
    c=0
    first_1 = labels.index(1)
    for i in range(len(tokenized_text)):
        if labels[i] == 1 and c==0 :
            test_seq_tokens.append(tokenized_text[i])
            test_seq_labels.append(labels[i])
            t = ''
        elif labels[i] == 1 and c!=0 :
            t = ' '.join(tokenized_text[s:i+1])
            test_seq_tokens.append(t)
            test_seq_labels.append(labels[i])
            s = i+1
            t = ''
        elif labels[i] == 0 and c == 0:
            s = i
            c = 1
        elif labels[i] == 0 and c != 0:
            t = ' '.join(tokenized_text[s:i+1])
            test_seq_tokens.append(t)
            test_seq_labels.append(labels[i])
        else:# labels[i] == 1 and c != 0:
            t = ' '.join(tokenized_text[s:i+1])
            test_seq_tokens.append(t)
            test_seq_labels.append(labels[i])
            s = 0
            c = 0
            t = ''
    return test_seq_tokens, test_seq_labels

In [ ]:
def datasets_for_training():
    #Splitting sequences and lables for traning and evaluating models
    #
    x_train, x_test1, y_train, y_test1 = train_test_split(sequences, labels, test_size=0.4)
    x_test1, x_test2, y_test1, y_test2 = train_test_split(x_test1,y_test1, test_size=0.5)

    d = {'train':Dataset.from_dict({'label':y_train,'text':x_train}),
     'test1':Dataset.from_dict({'label':y_test1,'text':x_test1}),
     'test2':Dataset.from_dict({'label':y_test2,'text':x_test2})
     }

    dataset = DatasetDict(d)
    return dataset

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=400)

In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
def training_model():
    uni_labels = ['Continue', "Change"]
    sequence_clf_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2)
    sequence_clf_model.config.id2label = {i: l for i, l in enumerate(uni_labels)}
    sequence_clf_model.config.dropout =0.2

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=2,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.05,
        logging_dir='./logs',
        load_best_model_at_end=True,
        logging_steps=1,
        log_level='info',
        evaluation_strategy='epoch',
        eval_steps=100,
        save_strategy='epoch'
    )

    trainer1 = Trainer(
        model=sequence_clf_model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['test1'],
        compute_metrics=compute_metrics,
        data_collator=data_collator
    )

    trainer2 = Trainer(
        model=sequence_clf_model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['test2'],
        compute_metrics=compute_metrics,
        data_collator=data_collator
    )

    #trainer1.evaluate()
    #tarining modesl with preprocessed dataset
    trainer1.train()
    #evaluation trained model
    #trainer1.evaluate()
    #evaluation model with another dataset
    #trainer1.evaluate()
    trainer1.save_model('./model/bert_distilbert_seq_token')

In [ ]:
if __name__ == '__main__':
    sequence_clf_model = None
    #============================================
    #Generate a df from Data class from data_load
    #A df contains "texc" column a sentence per a row and their lables(0: continue, 1: change)
    path ='hotels.json'
    data = load_dataset(path)
    print("\n\nInitial Data")
    display(data.df[:3])
    print(data.df.shape)
    print(data.df['label'].value_counts())
    #save initial df as a json file
    #df = data.df.to_json("df.json")

    #============================================
    #Preprocessing data for modeling
    #t = df.copy()
    t = data.df.copy()
    t = t[:4000]
    #4000 rows were used which will generate sequences.
    print("\n\nDataset to be used for modeling")
    print(t.shape)
    display(t[:5])
    #Converting columns to list for tokenizing
    test_text = t['text'].values
    test_label = t['label'].values

    #Splitting setences to tokens and labeling tokens
    text, tokenized_text, labels = split_token_sentences(test_text)
    labels = flatten_list(labels)
    tokenized_text = flatten_list(tokenized_text)
    print("\n\nSentences")
    print(text[:10])
    print("Tokenized sentences")
    print(tokenized_text[:20])
    print("Labels")
    print(labels[:20])

    #Generating sequnces to be used for training by combining tokens and labeling the sequences
    sequences, labels = generate_test_dataset(tokenized_text, labels)
    print("\n\nGenerated Sequences")
    print(sequences[:10])
    print("Lables for each sequences")
    print(labels[:10])
    print("The number of generated Sequences")
    print(len(sequences))

    #Generating datasets for training, vaildating and testing
    dataset =datasets_for_training()
    print("\n\nDatasets for training and evaluating models")
    print(dataset, end="\n\n")

    #Applying DistilBertTokenizerFast for DistilBert model
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', ignore_mismatched_sizes=True)
    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    print("\n\nConverted Sequences")
    print(tokenized_dataset['train'][15], end="\n\n")

    #padding dataset
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    #training and saving models
    training_model()

                  text  label  count
0               Hello!      0      1
1  How can I help you?      1      5
2                  Hi,      0      1


Initial Data


,text,label,count
0,Hello!,0,1
1,How can I help you?,1,5
2,"Hi,",0,1


(105030, 3)
0    53659
1    51371
Name: label, dtype: int64


Dataset to be used for modeling
(4000, 3)


,text,label,count
0,Hello!,0,1
1,How can I help you?,1,5
2,"Hi,",0,1
3,I would like to find a hotel.,1,7
4,Okay.,0,1




Sentences
['Hello!', 'How can I help you?', 'Hi,', 'I would like to find a hotel.', 'Okay.', 'else can you tell me about the hotel?', 'Find a hotel near the beachfront.', 'Okay beachfront in which town?', 'Oxnard California.', 'Okay.']
Tokenized sentences
['Hello!', 'How', 'can', 'I', 'help', 'you?', 'Hi,', 'I', 'would', 'like', 'to', 'find', 'a', 'hotel.', 'Okay.', 'else', 'can', 'you', 'tell', 'me']
Labels
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]


Generated Sequences
['Hello! How', 'Hello! How can', 'Hello! How can I', 'Hello! How can I help', 'Hello! How can I help you?', 'Hi,', 'Hi, I', 'Hi, I would', 'Hi, I would like', 'Hi, I would like to']
Lables for each sequences
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
The number of generated Sequences
18056


Datasets for training and evaluating models
DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 10833
    })
    test1: Dataset({
        features: ['label', 'text'],
        num_rows:

...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.22.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin




Converted Sequences
{'label': 0, 'text': "I'm going to need a balcony and", 'input_ids': [101, 1045, 1005, 1049, 2183, 2000, 2342, 1037, 11673, 1998, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

Epoch,Training Loss,Validation Loss,Accuracy
1,0.001800,0.094056,0.956799
2,2.873700,0.092126,0.959845


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3611
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-678
Configuration saved in ./results/checkpoint-678/config.json
Model weights saved in ./results/checkpoint-678/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3611
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1356
Configuration saved in ./results/checkpoint-1356/config.json
Model weights saved in ./r